In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/329T/Project/

[Errno 2] No such file or directory: '/content/drive/Shareddrives/329T/Project/'
/content


In [ ]:
%cd /content/drive/MyDrive/329T Project/

/content/drive/MyDrive/329T Project


In [ ]:
# Install packages 
!pip3 install transformers 
!pip3 install torch

In [ ]:
import warnings
import os

import torch
import torch.nn as nn
import json
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
from torch.nn.functional import cosine_similarity
from transformers import BertConfig, BertTokenizer, AutoConfig, AutoTokenizer, DistilBertConfig, DistilBertTokenizer
from transformers import BertForSequenceClassification, BertForMaskedLM, AutoModelForSequenceClassification, DistilBertForMaskedLM
import copy
import argparse
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import re
import copy

In [ ]:
import sys
import importlib
import subprocess
from pathlib import Path

def install_if_not_installed(packages):
  """Install the given packages if they are not already installed."""

  for package in packages:
    if isinstance(package, tuple):
      package_name, package_package = package
    else:
      package_name = package
      package_package = package

    print(f"{package_name} ... ", end='')

    try:
      importlib.import_module(package_name)
      print("already installed")

    except:
      print(f"installing from {package_package}")
      subprocess.check_call(
          [sys.executable, "-m", "pip", "install", package_package]
      )

install_if_not_installed(
    [("trulens", "git+https://github.com/truera/trulens.git@piotrm/vis/output-detect"),
     "pandas",
     "numpy",
     "domonic"]
)

trulens ... already installed
pandas ... already installed
numpy ... already installed
domonic ... already installed


In [ ]:
from trulens.nn.attribution import IntegratedGradients
from trulens.nn.models import get_model_wrapper
from trulens.nn.attribution import Cut, OutputCut
from trulens.nn.quantities import MaxClassQoI, ClassQoI, ComparativeQoI
from torch.nn.functional import cosine_similarity
from trulens.visualizations import NLP

##### Args

In [ ]:
device = 'cuda'
data_path = "tweet_sentiment_pairs.csv"
mlm_path = "distilbert-base-uncased"
tgt_path = "distilbert-base-uncased-finetuned-sst-2-english"
output_dir = "perturbations.csv"
use_sim_mat = 0 
start = 0
end = 100
num_label = 2 
use_bpe = 1
k = 5
threshold_pred_score = 0

##### Initialize models

In [ ]:
tokenizer_mlm = DistilBertTokenizer.from_pretrained(mlm_path, do_lower_case=True)
tokenizer_tgt = AutoTokenizer.from_pretrained(tgt_path, do_lower_case=True)

config_atk = DistilBertConfig.from_pretrained(mlm_path)
mlm_model = DistilBertForMaskedLM.from_pretrained(mlm_path, config=config_atk)
mlm_model.to(device)

config_tgt = AutoConfig.from_pretrained(tgt_path, num_labels=num_label)
tgt_model = AutoModelForSequenceClassification.from_pretrained(tgt_path, config=config_tgt)
tgt_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# For pytorch, input_shape is required... I just used the tokenizer max_length here
wrapped_model = get_model_wrapper(tgt_model, input_shape=(None, 128), device=device)

INFO: Detected pytorch backend for <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>.
INFO: Using backend Backend.PYTORCH.
INFO: If this seems incorrect, you can force the correct backend by passing the `backend` parameter directly into your get_model_wrapper call.


### Integrated Gradients

In [ ]:
def tokenize(inputs):
    return tokenizer_tgt(inputs, padding=True, return_tensors="pt").to(device)
    # pt refers to pytorch tensor

In [ ]:
# Baseline 
from trulens.utils.nlp import token_baseline

inputs_baseline_ids, inputs_baseline_embeddings = token_baseline(
    keep_tokens=set([tokenizer_tgt.cls_token_id, tokenizer_tgt.sep_token_id]),
    # Which tokens to preserve.

    replacement_token=tokenizer_tgt.pad_token_id,
    # What to replace tokens with.

    input_accessor=lambda x: x.kwargs['input_ids'],

    ids_to_embeddings=tgt_model.get_input_embeddings()
    # Callable to produce embeddings from token ids.
)


# Provisional baseline for calling .attributions as opposed to visualizing with NLP object
inputs_baseline_ids_attribution, inputs_baseline_embeddings_attribution = token_baseline(
    keep_tokens=set([tokenizer_tgt.cls_token_id, tokenizer_tgt.sep_token_id]),
    # Which tokens to preserve.

    replacement_token=tokenizer_tgt.pad_token_id,
    # What to replace tokens with.

    input_accessor=lambda x: x.args[0],

    ids_to_embeddings=tgt_model.get_input_embeddings()
    # Callable to produce embeddings from token ids.
)

In [ ]:
def similarity_metrics(x,y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    diff = x - y
    return {
        'L_1': torch.norm(diff, p=1).item(),
        'L_2': torch.norm(diff, p=2).item(),
        'L_inf': torch.max(torch.abs(diff)).item(),
        'cosine_sim': cosine_similarity(x,y,dim=0).item()
    }

IG_calc = IntegratedGradients(
    wrapped_model, 
    baseline=inputs_baseline_embeddings, 
    resolution=50,
    doi_cut=Cut('distilbert_embeddings_word_embeddings'), 
    qoi=ComparativeQoI(1, 0), #qoi='max',
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)

IG_calc_attribution = IntegratedGradients(
    wrapped_model, 
    baseline=inputs_baseline_embeddings_attribution, 
    resolution=50,
    doi_cut=Cut('distilbert_embeddings_word_embeddings'), 
    qoi=ComparativeQoI(1, 0), #qoi='max',  
    qoi_cut=OutputCut(accessor=lambda o: o['logits'])
)

In [ ]:
'''
Input:  integrated gradient calculator: (trulens.nn.attribution.IntegratedGradients())
        original text: (str)
        perturbed text: (str)
Output: attribution for original text: (tensor of shape (?))
        attribution for perturbed text: (tensor of shape (?))
Function:
        encode raw text of an original-adversarial pair into input tokens for the distilbert model
        feed through IG
        compute metrics
'''
def calculate_IGs(IG, string):
    encode_string = tokenize(string)
    input_ids = torch.tensor(encode_string["input_ids"])
    attribution = IG.attributions(input_ids.to(device))
    
    return attribution

### Attack code from BERT-Attack

In [ ]:
device = 'cuda'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore', category=FutureWarning)

filter_words = ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'ain', 'all', 'almost',
                'alone', 'along', 'already', 'also', 'although', 'am', 'among', 'amongst', 'an', 'and', 'another',
                'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'aren', "aren't", 'around', 'as',
                'at', 'back', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides',
                'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot', 'could', 'couldn', "couldn't", 'd', 'didn',
                "didn't", 'doesn', "doesn't", 'don', "don't", 'down', 'due', 'during', 'either', 'else', 'elsewhere',
                'empty', 'enough', 'even', 'ever', 'everyone', 'everything', 'everywhere', 'except', 'first', 'for',
                'former', 'formerly', 'from', 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'he', 'hence',
                'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his',
                'how', 'however', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'isn', "isn't", 'it', "it's",
                'its', 'itself', 'just', 'latter', 'latterly', 'least', 'll', 'may', 'me', 'meanwhile', 'mightn',
                "mightn't", 'mine', 'more', 'moreover', 'most', 'mostly', 'must', 'mustn', "mustn't", 'my', 'myself',
                'namely', 'needn', "needn't", 'neither', 'never', 'nevertheless', 'next', 'no', 'nobody', 'none',
                'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'o', 'of', 'off', 'on', 'once', 'one', 'only',
                'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'per',
                'please', 's', 'same', 'shan', "shan't", 'she', "she's", "should've", 'shouldn', "shouldn't", 'somehow',
                'something', 'sometime', 'somewhere', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs',
                'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein',
                'thereupon', 'these', 'they', 'this', 'those', 'through', 'throughout', 'thru', 'thus', 'to', 'too',
                'toward', 'towards', 'under', 'unless', 'until', 'up', 'upon', 'used', 've', 'was', 'wasn', "wasn't",
                'we', 'were', 'weren', "weren't", 'what', 'whatever', 'when', 'whence', 'whenever', 'where',
                'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while',
                'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'with', 'within', 'without', 'won',
                "won't", 'would', 'wouldn', "wouldn't", 'y', 'yet', 'you', "you'd", "you'll", "you're", "you've",
                'your', 'yours', 'yourself', 'yourselves', '-', ',', '.', ':', ';', '(', ')', '"', "'", '|', '...']
filter_words = set(filter_words)


def get_sim_embed(embed_path, sim_path):
    id2word = {}
    word2id = {}

    with open(embed_path, 'r', encoding='utf-8') as ifile:
        for line in ifile:
            word = line.split()[0]
            if word not in id2word:
                id2word[len(id2word)] = word
                word2id[word] = len(id2word) - 1

    cos_sim = np.load(sim_path)
    return cos_sim, word2id, id2word



class Feature(object):
    def __init__(self, seq_a, label, example_index):
        self.example_index = example_index
        self.label = label
        self.pred = -1
        self.seq = seq_a
        self.final_adverse = seq_a
        self.query = 0
        self.change = 0
        self.success = 0
        self.sim = 0.0
        self.metrics = None
        self.changes = []
        self.attributions = None


def _tokenize(seq, tokenizer):
    seq = seq.replace('\n', '').lower()
    words = seq.split(' ')

    sub_words = []
    keys = []
    index = 0
    for word in words:
        sub = tokenizer.tokenize(word)
        sub_words += sub
        keys.append([index, index + len(sub)])
        index += len(sub)

    return words, sub_words, keys


def _get_masked(words):
    len_text = len(words)
    masked_words = []
    for i in range(len_text - 1):
        masked_words.append(words[0:i] + ['[UNK]'] + words[i + 1:])
    # list of words
    return masked_words


def get_important_scores(words, tgt_model, orig_prob, orig_label, orig_probs, tokenizer_tgt, batch_size, max_length):
    masked_words = _get_masked(words)
    texts = [' '.join(words) for words in masked_words]  # list of text of masked words
    all_input_ids = []
    all_masks = []
    all_segs = []
    for text in texts:
        inputs = tokenizer_tgt.encode_plus(text, None, add_special_tokens=True, max_length=max_length, return_token_type_ids=True, truncation=True)
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
        attention_mask = [1] * len(input_ids)
        padding_length = max_length - len(input_ids)
        input_ids = input_ids + (padding_length * [0])
        token_type_ids = token_type_ids + (padding_length * [0])
        attention_mask = attention_mask + (padding_length * [0])
        all_input_ids.append(input_ids)
        all_masks.append(attention_mask)
        all_segs.append(token_type_ids)
    seqs = torch.tensor(all_input_ids, dtype=torch.long)
    masks = torch.tensor(all_masks, dtype=torch.long)
    segs = torch.tensor(all_segs, dtype=torch.long)
    seqs = seqs.to(device)

    eval_data = TensorDataset(seqs)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=batch_size)
    leave_1_probs = []
    for batch in eval_dataloader:
        masked_input, = batch
        bs = masked_input.size(0)

        leave_1_prob_batch = tgt_model(masked_input)[0]  # B num-label
        leave_1_probs.append(leave_1_prob_batch)
    if len(leave_1_probs) == 0:
      return []

    leave_1_probs = torch.cat(leave_1_probs, dim=0)  # words, num-label
    leave_1_probs = torch.softmax(leave_1_probs, -1)  #
    leave_1_probs_argmax = torch.argmax(leave_1_probs, dim=-1)
    import_scores = (orig_prob
                     - leave_1_probs[:, orig_label]
                     +
                     (leave_1_probs_argmax != orig_label).float()
                     * (leave_1_probs.max(dim=-1)[0] - torch.index_select(orig_probs, 0, leave_1_probs_argmax))
                     ).data.cpu().numpy()

    return import_scores


def get_substitues(substitutes, tokenizer, mlm_model, use_bpe, substitutes_score=None, threshold=3.0):
    # substitues L,k
    # from this matrix to recover a word
    words = []
    sub_len, k = substitutes.size()  # sub-len, k

    if sub_len == 0:
        return words

    elif sub_len == 1:
        for (i,j) in zip(substitutes[0], substitutes_score[0]):
            if threshold != 0 and j < threshold:
                break
            words.append(tokenizer._convert_id_to_token(int(i)))

    return words


def get_bpe_substitues(substitutes, tokenizer, mlm_model):
    # substitutes L, k

    substitutes = substitutes[0:5, 0:4] # maximum BPE candidates # 0:12

    # find all possible candidates

    all_substitutes = []
    for i in range(substitutes.size(0)):
        if len(all_substitutes) == 0:
            lev_i = substitutes[i]
            all_substitutes = [[int(c)] for c in lev_i]
        else:
            lev_i = []
            for all_sub in all_substitutes:
                for j in substitutes[i]:
                    lev_i.append(all_sub + [int(j)])
            all_substitutes = lev_i

    # all substitutes  list of list of token-id (all candidates)
    c_loss = nn.CrossEntropyLoss(reduction='none')
    word_list = []
    # all_substitutes = all_substitutes[:24]
    all_substitutes = torch.tensor(all_substitutes) # [ N, L ]
    all_substitutes = all_substitutes[:24].to(device)
    N, L = all_substitutes.size()
    word_predictions = mlm_model(all_substitutes)[0] # N L vocab-size
    ppl = c_loss(word_predictions.view(N*L, -1), all_substitutes.view(-1)) # [ N*L ]
    ppl = torch.exp(torch.mean(ppl.view(N, L), dim=-1)) # N
    _, word_list = torch.sort(ppl)
    word_list = [all_substitutes[i] for i in word_list]
    final_words = []
    for word in word_list:
        tokens = [tokenizer._convert_id_to_token(int(i)) for i in word]
        text = tokenizer.convert_tokens_to_string(tokens)
        final_words.append(text)
    return final_words

def num_tokens(seq):
  inputs = tokenizer_tgt.encode_plus(seq, None, add_special_tokens=True, max_length=128, return_token_type_ids=True, truncation=True)
  return torch.tensor(inputs["input_ids"]).size(0)

def top_k_agree(original_attribuion, perturbed_attribution, k=5):
  original_magnitudes = np.linalg.norm(original_attribution, ord=1, dim=-1)
  perturbed_magnitudes = np.linalg.norm(perturbed_attribution, ord=1, dim=-1)
  original_indices = np.argpartition(original_magnitudes, -k)[-k:]  # Indices not sorted
  perturbed_indices = np.argpartition(perturbed_magnitudes, -k)[-k:]  # Indices not sorted
  return original_indices[np.argsort(original_magnitudes[original_indices])][::-1] == perturbed_indices[np.argsort(perturbed_magnitudes[perturbed_indices])][::-1]

def attack(feature, tgt_model, mlm_model, tokenizer_tgt, tokenizer_mlm, k, batch_size, max_length=128, cos_mat=None, w2i={}, i2w={}, use_bpe=1, threshold_pred_score=0.3):
    features = []

    # Original attribution
    original_attribution = calculate_IGs(IG_calc_attribution, feature.seq)
    original_attr_summed = torch.sum(original_attribution, dim=-1).flatten()
    original_attr_summed_np = list(original_attr_summed.cpu().numpy())

    # Get words, subwords, and the indices corresponding to that word in the tokenized sequence
    words, sub_words, keys = _tokenize(feature.seq, tokenizer_tgt)

    # Get original label (orig_label) and current probability (current_prob)
    # Feed in original sequence (feature.seq) into target model
    inputs = tokenizer_tgt.encode_plus(feature.seq, None, add_special_tokens=True, max_length=max_length, return_token_type_ids=True, truncation=True)
    input_ids, token_type_ids = torch.tensor(inputs["input_ids"]), torch.tensor(inputs["token_type_ids"])
    attention_mask = torch.tensor([1] * len(input_ids))
    seq_len = input_ids.size(0)
    num_tokens_original = seq_len
    with torch.no_grad():
      orig_probs = tgt_model(input_ids.unsqueeze(0).to(device),
                            attention_mask=attention_mask.unsqueeze(0).to(device),
                            # token_type_ids=token_type_ids.unsqueeze(0).to(device)
                            )[0].squeeze()
    orig_probs = torch.softmax(orig_probs, -1)
    orig_label = torch.argmax(orig_probs)
    current_prob = orig_probs.max()

    # Feed subwords into MLM model
    sub_words = ['[CLS]'] + sub_words[:max_length - 2] + ['[SEP]']
    input_ids_ = torch.tensor([tokenizer_mlm.convert_tokens_to_ids(sub_words)])
    # For each subword in the sequence, outputs a prediction (vocab size vector)
    with torch.no_grad():
        word_predictions = mlm_model(input_ids_.to(device))[0].squeeze()  # seq-len(sub) vocab
    # word_pred_scores_all is the top k values, word_predictions is their corresponding indices
    word_pred_scores_all, word_predictions = torch.topk(word_predictions, k, -1)  # seq-len k
    # Top k predictions (indices) for subwords (other than CLS and SEP)
    word_predictions = word_predictions[1:len(sub_words) + 1, :]
    # Top k predictions (probabilities) for subwords (other than CLS and SEP)
    word_pred_scores_all = word_pred_scores_all[1:len(sub_words) + 1, :]
    # Get importance scores for each word
    important_scores = get_important_scores(words, tgt_model, current_prob, orig_label, orig_probs,
                                            tokenizer_tgt, batch_size, max_length)
    # Store number of words??
    feature.query += int(len(words))
    # Get indices of words in decreasing importance score order
    list_of_index = sorted(enumerate(important_scores), key=lambda x: x[1], reverse=True)
    final_words = copy.deepcopy(words)

    # Iterate through words in decreasing importance score order
    for top_index in list_of_index:
        # If number of changes exceeds 40% of the whole sequence, return
        if feature.change > int(0.4 * (len(words))): # 0.4 is the perturbation percentage
            return None

        # tgt_word is the word to replace 
        tgt_word = words[top_index[0]]
        if tgt_word in filter_words:
            continue
        if keys[top_index[0]][0] > max_length - 2:
            continue

        # For each of L tokens in the word, top k predictions
        substitutes = word_predictions[keys[top_index[0]][0]:keys[top_index[0]][1]]  # L, k
        word_pred_scores = word_pred_scores_all[keys[top_index[0]][0]:keys[top_index[0]][1]]
        # Getting most likely substitutes based on bpe
        substitutes = get_substitues(substitutes, tokenizer_mlm, mlm_model, use_bpe, word_pred_scores, threshold_pred_score)

        least_similarity  = 1.0
        candidate = None
        for substitute_ in substitutes:
            substitute = substitute_

            if substitute == tgt_word:
                continue  # filter out original word
            if '##' in substitute:
                continue  # filter out sub-word

            if substitute in filter_words:
                continue
            if substitute in w2i and tgt_word in w2i:
                if cos_mat[w2i[substitute]][w2i[tgt_word]] < 0.4:
                    continue


            temp_replace = final_words
            # Replace word with with substitute 
            temp_replace[top_index[0]] = substitute
            # Convert back to text
            temp_text = tokenizer_tgt.convert_tokens_to_string(temp_replace)
            # Tokenize with target model tokenizer
            inputs = tokenizer_tgt.encode_plus(temp_text, None, add_special_tokens=True, max_length=max_length, )
            # Feed into target model
            input_ids = torch.tensor(inputs["input_ids"]).unsqueeze(0).to(device)
            seq_len = input_ids.size(1)
            with torch.no_grad():
                temp_prob = tgt_model(input_ids)[0].squeeze()
            # Increment query 
            feature.query += 1
            # Get output label
            temp_prob = torch.softmax(temp_prob, -1)
            temp_label = torch.argmax(temp_prob)
            feature.pred = temp_label.detach().cpu().numpy()

            # Skip if number of tokens don't match 
            if num_tokens(temp_text) != num_tokens_original:
              continue

            # Calculate integrated gradients and similarity between them
            perturbed_attribution = calculate_IGs(IG_calc_attribution, temp_text)
            perturbed_attr_summed= torch.sum(perturbed_attribution, dim=-1).flatten()
            feature.metrics = similarity_metrics(original_attr_summed, perturbed_attr_summed)
            feature.attributions = original_attr_summed_np, list(perturbed_attr_summed.cpu().numpy())

            if feature.metrics['cosine_sim'] < 0.863:
                # If succeeded, increment change
                feature.change += 1
                # Make the change
                final_words[top_index[0]] = substitute
                # Append the actual changes to changes
                feature.changes.append([keys[top_index[0]][0], substitute, tgt_word])
                # Store the adversarial example text in final_adverse 
                feature.final_adverse = temp_text
                feature.success = 4
                return feature
            else:
                # Store the example with the least similarity
                if feature.metrics['cosine_sim'] < least_similarity:
                    least_similarity = feature.metrics['cosine_sim']
                    candidate = substitute
                feature.final_adverse = temp_text

        if least_similarity < 1.0:
            feature.change += 1
            feature.changes.append([keys[top_index[0]][0], candidate, tgt_word])
            final_words[top_index[0]] = candidate

    return None

##### Load data

In [ ]:
def preprocess_df(df):
  df['text'] = df['text'].apply(lambda x: re.sub(r'http[^\s]*', '', x))
  df['text'] = df['text'].apply(lambda x: re.sub('@[^\s]+','', x))
  df['text'] = df['text'].apply(lambda x: re.sub(r'#', '', x))
  df['text'] = df['text'].apply(lambda x: re.sub(r'-', '', x))
  df['text'] = df['text'].apply(lambda x: x.strip())
  return df

In [ ]:
def get_data_cls(data_path):
    df = pd.read_csv(data_path)
    df = preprocess_df(df)
    features = []
    seqs = list(df['text'])
    labels = list(df['sentiment'])
    features = list(zip(seqs, labels))
    return features 
    
features = get_data_cls(data_path)
print(features[:5])

[("Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", 0), ("is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0), ('I dived many times for the ball. Managed to save 50%  The rest go out of bounds', 0), ('my whole body feels itchy and like its on fire', 0), ("no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.", 0)]


##### Generate adversarial examples

In [ ]:
if use_sim_mat == 1:
    cos_mat, w2i, i2w = get_sim_embed('data_defense/counter-fitted-vectors.txt', 'data_defense/cos_sim_counter_fitting.npy')
else:
    cos_mat, w2i, i2w = None, {}, {}

features_output = []

# old_time = time.time()

start = 0 
end = 5000
outputs = []

for index, feature in enumerate(tqdm(features[start:end])):
    seq_a, label = feature
    feat = Feature(seq_a, label, index + start)

    # print('\r number {:d} '.format(index) + tgt_path, end='')
    feature = attack(feat, tgt_model, mlm_model, tokenizer_tgt, tokenizer_mlm, k, batch_size=32, max_length=128,
                  cos_mat=cos_mat, w2i=w2i, i2w=i2w, use_bpe=use_bpe,threshold_pred_score=threshold_pred_score)

    if feature:
        outputs.append({'example_index': feature.example_index,
                        'label': feature.label,
                        'prediction': feature.pred,
                        'success': feature.success,
                        'change': feature.change,
                        'num_word': len(feature.seq.split(' ')),
                        'query': feature.query,
                        'changes': feature.changes,
                        'seq_a': feature.seq,
                        'adv': feature.final_adverse,
                        'original_attribution': feature.attributions[0],
                        'perturbed_attribution': feature.attributions[1],
                        'cosine_similarity': feature.metrics['cosine_sim'],
                        'L1': feature.metrics['L_1'],
                        'L2': feature.metrics['L_2'],
                        'Linf': feature.metrics['L_inf']
                        })
        
    if index % 100 == 0:
      df = pd.DataFrame(outputs)
      df.to_csv('adversarial_examples_cosine.csv')

100%|██████████| 5000/5000 [1:06:26<00:00,  1.25it/s]


##### Save generated examples to csv

In [ ]:
output_dir = "adversarial_examples_cosine.csv"

def dump_features(features):
    outputs = []

    for feature in features:
        if feature:
            outputs.append({'label': feature.label,
                            'prediction': feature.pred,
                            'success': feature.success,
                            'change': feature.change,
                            'num_word': len(feature.seq.split(' ')),
                            'query': feature.query,
                            'changes': feature.changes,
                            'seq_a': feature.seq,
                            'adv': feature.final_adverse,
                            'original_attribution': feature.attributions[0],
                            'perturbed_attribution': feature.attributions[1],
                            'cosine_similarity': feature.metrics['cosine_sim'],
                            'L1': feature.metrics['L_1'],
                            'L2': feature.metrics['L_2'],
                            'Linf': feature.metrics['L_inf']
                            })

    df = pd.DataFrame(outputs)
    df.to_csv(output_dir)
    print('finished dump')
    return df

df = dump_features(features_output[:(9000 - 5041)])

finished dump


In [ ]:
df = pd.read_csv('adversarial_examples_cosine.csv')
len(df) / max(list(df['example_index']))

### Analysis

In [ ]:
!pip install pyinflect 
!pip install -U spacy
!python -m spacy download en_core_web_sm
import nltk
nltk.download('wordnet')

     |████████████████████████████████| 703 kB 5.2 MB/s 
     |████████████████████████████████| 6.2 MB 5.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 660 kB 34.3 MB/s 
     |████████████████████████████████| 457 kB 53.6 MB/s 
     |████████████████████████████████| 181 kB 46.8 MB/s 
     |████████████████████████████████| 10.1 MB 38.4 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-ope

     |████████████████████████████████| 12.8 MB 5.3 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import pyinflect
import spacy
import ast 
from nltk.corpus import wordnet

nlp = spacy.load("en_core_web_sm")

def tense_change(original_token, perturbed_token):
	original_token = nlp(original_token)[0]
	perturbed_token = nlp(perturbed_token)[0]

	if original_token.tag_ in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
		if perturbed_token.tag_ in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
			if original_token._.inflect("VB") == perturbed_token._.inflect("VB"):
				return True 
	return False

def semantic_similarity(original_token, perturbed_token):
	original_token = nlp(original_token)[0]
	perturbed_token = nlp(perturbed_token)[0]

	return original_token.similarity(perturbed_token)

def is_synonym(original_token, perturbed_token):
  perturbed_token_lemma = nlp(perturbed_token)[0].lemma_
  for synset in wordnet.synsets(original_token):
    for lemma in synset.lemma_names():
      if perturbed_token_lemma == lemma:
        return True
  return False

In [ ]:
# Postprocess data with different heuristics 
all_tense_changes = []
all_semantic_similarities = []
all_synonyms = []

for i, d in tqdm(df.iterrows(), total=df.shape[0]):
  # if i % 100 == 0:
  #   print("i", i)
  tense_changes = []
  semantic_similarities = []
  synonyms = []
  for change in ast.literal_eval(d['changes']):
    token_index, perturbed, original = change
    tense_changed = tense_change(original, perturbed)
    tense_changes.append(1 if tense_changed else 0)
    semantic_similarities.append(semantic_similarity(original, perturbed))
    synonyms.append(1 if is_synonym(original, perturbed) and (not tense_changed) else 0)
  assert len(ast.literal_eval(d['changes'])) == len(tense_changes)
  assert len(ast.literal_eval(d['changes'])) == len(semantic_similarities)
  assert len(ast.literal_eval(d['changes'])) == len(synonyms)
  all_tense_changes.append(tense_changes)
  all_semantic_similarities.append(semantic_similarities)
  all_synonyms.append(synonyms)

df['tense_change'] = all_tense_changes
df['semantic_similarity'] =all_semantic_similarities
df['synonym'] = all_synonyms

100%|██████████| 6206/6206 [07:58<00:00, 12.98it/s]


In [ ]:
df.to_csv('adversarial_examples_cosine_postprocessed.csv')

Number of generated adversarial examples: 7


In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
bounds = np.linspace(0.137, 2, num=31)
df['bin'] = np.digitize(np.array(df['cosine_distance']), bounds, right=True)

In [ ]:
df['tense_change_none'] = df['tense_change'].apply(lambda x: np.all(1-np.array(x)).astype(int)) 
df['semantic_similarity_min'] = df['semantic_similarity'].apply(lambda x: min(x))  
df['semantic_similarity_max'] = df['semantic_similarity'].apply(lambda x: max(x))  
df['semantic_similarity_mean'] = df['semantic_similarity'].apply(lambda x: np.mean(x))  
df['semantic_similarity_median'] = df['semantic_similarity'].apply(lambda x: np.median(x))  
df['synonym_all'] = df['synonym'].apply(lambda x: np.all(x).astype(int))  
df_agg = df.groupby('bin').agg({'tense_change_none': 'mean', 'synonym_all': 'mean', 'semantic_similarity_mean': 'mean', 'cosine_distance': 'mean'})
df_agg['bin'] = df_agg.index

In [ ]:
def colors_from_values(values, palette_name):
    # normalize the values to range [0, 1]
    normalized = (values - min(values)) / (max(values) - min(values))
    # convert to indices
    indices = np.round(normalized * (len(values) - 1)).astype(np.int32)
    # use the indices to get the colors
    palette = sns.color_palette(palette_name, len(values))
    return np.array(palette).take(indices, axis=0)

In [ ]:
sns.set_context('poster')
plt.figure(figsize=(12, 4))
ax = sns.barplot(x='bin', y='tense_change_none', data=df_agg, palette=colors_from_values(df_agg['tense_change_none'], "magma"))
ax.set(xticklabels=[])
ax.set_ylim(0.7, 1)
ax.set_xlabel('Discretized dissimilarity between \n attributions (cosine distance)')
ax.set_ylabel('% of example \n pairs with \n consistent tense')
ax.xaxis.label.set_color('#cc0000')
ax.yaxis.label.set_color('#3c78d8')
ax.tick_params(bottom=False)


In [ ]:
plt.figure(figsize=(12, 4))
ax = sns.barplot(x='bin', y='synonym_all', data=df_agg, palette=colors_from_values(df_agg['synonym_all'], "magma"))
ax.set(xticklabels=[])
ax.set_ylim(0, 0.2)
ax.set_xlabel('Discretized dissimilarity between \n attributions (cosine distance)')
ax.xaxis.label.set_color('#cc0000')
ax.yaxis.label.set_color('#3c78d8')
ax.set_ylabel('% of example pairs \n with only synonym \n substitutions')
ax.tick_params(bottom=False)

##### Check accuracy of target model

In [ ]:
num_correct = 0
for seq, label in features[start:end]: 
  inputs = tokenizer_tgt.encode_plus(seq, None, add_special_tokens=True, max_length=128, return_token_type_ids=True, truncation=True)
  input_ids, token_type_ids = torch.tensor(inputs["input_ids"]), torch.tensor(inputs["token_type_ids"])
  attention_mask = torch.tensor([1] * len(input_ids))
  seq_len = input_ids.size(0)
  probs = tgt_model(input_ids.unsqueeze(0).to(device),
                          attention_mask=attention_mask.unsqueeze(0).to(device),
                          )[0].squeeze()
  probs = torch.softmax(probs, -1)
  pred_label = torch.argmax(probs)
  if label == pred_label:
    num_correct += 1 

print("Accuracy: {}".format(num_correct / len(features[start:end])))


##### Old code for running `bertattack.py`

In [ ]:
import subprocess    

In [ ]:
run = f"python3 bertattack.py --data_path tweet_sentiment_pairs.csv\
                                                 --mlm_path bert-base-uncased\
                                                 --tgt_path siebert/sentiment-roberta-large-english\
                                                  --use_sim_mat 0\
                                                  --output_dir data_defense/imdb_logs.tsv\
                                                  --num_label 2\
                                                  --use_bpe 1\
                                                  --k 48\
                                                  --start 0\
                                                  --end 1000\
                                                  --threshold_pred_score 0"

In [ ]:
!{run}

start process
Downloading: 100% 226k/226k [00:00<00:00, 272kB/s]
Downloading: 100% 28.0/28.0 [00:00<00:00, 18.2kB/s]
Downloading: 100% 570/570 [00:00<00:00, 532kB/s]
Downloading: 100% 256/256 [00:00<00:00, 242kB/s]
Downloading: 100% 687/687 [00:00<00:00, 574kB/s]
Downloading: 100% 780k/780k [00:01<00:00, 738kB/s] 
Downloading: 100% 446k/446k [00:01<00:00, 422kB/s]
Downloading: 100% 150/150 [00:00<00:00, 116kB/s]
Downloading: 100% 420M/420M [00:06<00:00, 71.9MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identi